In [6]:
import os
import subprocess
import matplotlib.pyplot as plt

In [7]:
gpu = 'ga_gpu_output.txt'
cpu = 'ga_cpu_output.txt'
tsp_dir = os.getcwd() + os.sep + 'TSPLIB'

In [8]:
tsp_files_list = os.listdir(tsp_dir)

In [19]:
def execGA(device = 'gpu', filename = 'gr17.tsp', p = 50, g = 50, m = 10):
    extraArgs = ['p', str(p), 'g', str(g), 'm', str(m)]
    if(device == 'gpu'):
        return subprocess.run(["./ga_gpu", filename] + extraArgs, capture_output=True)
    else:
        return subprocess.run(["./ga_cpu", filename] + extraArgs, capture_output=True)

In [10]:
def parseOutputFile(filename):
    fo = open(filename, 'r')
    lines = fo.readlines()
    num_gen = int(lines[0])
    res = {}
    res['g'] = num_gen
    solutions = []
    for i in range(num_gen):
        solutions.append(float(lines[i+1]))
    res['solns'] = solutions
    conv = int(lines[num_gen+1])
    res['converged'] = conv
    time = float(lines[num_gen+2])
    res['time'] = time
    return res

In [12]:
def plotExecTime():
    

In [15]:
plotExecTime()

['ulysses16.tsp', 'att532.tsp', 'pr226.tsp', 'kroA200.tsp', 'u1060.tsp', 'rd100.tsp', 'pla7397.tsp', 'dsj1000.tsp', 'd493.tsp', 'd2103.tsp', 'ts225.tsp', 'pr144.tsp', 'p654.tsp', 'pr152.tsp', 'd15112.tsp', 'swiss42.tsp', 'pla33810.tsp', 'brg180.tsp', 'eil51.tsp', 'lin318.tsp', 'u724.tsp', 'gr120.tsp', 'rat99.tsp', 'xray.problems', 'pr299.tsp', 'si1032.tsp', 'brd14051.tsp', 'd18512.tsp', 'gr96.tsp', 'kroC100.tsp', 'gr48.tsp', 'kroA100.tsp', 'pa561.tsp', 'dantzig42.tsp', 'fl3795.tsp', 'kroB200.tsp', 'fnl4461.tsp', 'gr24.tsp', 'rat195.tsp', 'solutions', 'kroB100.tsp', 'a280.tsp', 'vm1084.tsp', 'gr666.tsp', 'pr2392.tsp', 'brazil58.tsp', 'gr431.tsp', 'gil262.tsp', 'bayg29.tsp', 'rat575.tsp', 'lin105.tsp', 'rl1304.tsp', 'pr136.tsp', 'rl1889.tsp', 'fl1577.tsp', 'bier127.tsp', 'u159.tsp', 'rat783.tsp', 'rl5934.tsp', 'kroD100.tsp', 'fl1400.tsp', 'u1817.tsp', 'kroE100.tsp', 'nrw1379.tsp', 'u2319.tsp', 'pcb1173.tsp', 'pr76.tsp', 'd1655.tsp', 'vm1748.tsp', 'README.md', 'ch130.tsp', 'd198.tsp', 'al

In [20]:
execGA()

CompletedProcess(args=['./ga_gpu', 'gr17.tsp', 'p', '50', 'g', '50', 'm', '10'], returncode=0, stdout=b'-------------- 0 --------------\nSolution: 2950.000000\n-------------- 1 --------------\nSolution: 2557.000000\n-------------- 2 --------------\nSolution: 2443.000000\n-------------- 3 --------------\nSolution: 2333.000000\n-------------- 4 --------------\nSolution: 2265.000000\n-------------- 5 --------------\nSolution: 2229.000000\n-------------- 6 --------------\nSolution: 2218.000000\n-------------- 7 --------------\nSolution: 2209.000000\n-------------- 8 --------------\nSolution: 2196.000000\n-------------- 9 --------------\nSolution: 2180.000000\n-------------- 10 --------------\nSolution: 2179.000000\n-------------- 11 --------------\nSolution: 2176.000000\n-------------- 12 --------------\nSolution: 2167.000000\n-------------- 13 --------------\nSolution: 2167.000000\n-------------- 14 --------------\nSolution: 2167.000000\n-------------- 15 --------------\nSolution: 2167.00